In [2]:
import numpy as np
from scipy.special import factorial

class ExpectedChi:
    
    def __init__(self, lam = 1.0, T = 5, sim = 10, random_state = 1):
        self.lam = lam # parameter of exponential distribution which rho's follow
        self.T = T # time
        self.sim = sim # simulation number
        self.random_state = random_state
    
    def setparam(self):
        # params[0, :]　：　rho_eta
        # params[(1,2,3,4), :] : rho_chi for each candidate
        # params[(5,6,7,8), :] : mu_chi for each candidate
        # params[(9,10,11,12), :] : chi for each candidate
        r = np.random.RandomState(self.random_state)
        rho = r.exponential(self.lam, (5, self.sim))
        mu_chi = r.normal(0.0,1.0,(8, self.sim))
        params = np.vstack((rho, mu_chi))
        return params
        
    def signals(self):
        # make the 4 dimentional array : all_signals
        # each element in all_signals is 3 dimentional array whose shape is (T! * 4 * T )
        params = self.setparam()
        all_signals = np.empty([self.sim, factorial(self.T).astype(int), 4, self.T])
        g = np.random.RandomState(self.random_state)
        for i in range(self.sim):
            for k in range(factorial(self.T).astype(int)):
                for j in range(4):
                    signal = g.normal(params[j + 9, i], 1/params[0, i],  (1, self.T))
                    all_signals[i, k, j, :] = signal
        return all_signals
    
    def cum_signals(self):
        # calculating cumulative signals for expected chi_k given omega
        sig = self.signals()
        for i in range(self.sim):
            for j in range(self.T):
                if j != 0:
                    sig[i, :, :, :][:, :, j] = sig[i, :, :, :][:, :, j] + sig[i, :, :, :][:, :, j -1]
        return sig
            
    def expected_chi(self):
        # calculating expected chi_k given omega
        cum = self.cum_signals()
        params = self.setparam()
        transform = np.empty((self.sim, factorial(self.T).astype(int), 4, self.T))
        
        # first multiply cum by rho_eta 
        for i in range(self.sim):
            transform[i, :, :, :] = cum[i, :, :, :] * params[0, i]
            
        # second add rho_chi + mu_chi to transform
        for i in range(self.sim):
            for j in range(4):
                transform[i, :, j, :] = transform[i, :, j, :] + (params[j + 1, i] * params[j + 5, i])
        
        # third divide by rho_chi + t * rho_eta
        for i in range(self.sim):
            for j in range(4):
                for t in range(self.T):
                    transform[i, :, j, t] = transform[i, :, j, t] / (params[j + 1, i] + t * params[0, i])
        
        return transform
    
    def means(self, each_sim = True):
        # calculate mean
        if each_sim == True:
            # calculate the means of expected chi for each parameter set and t
            # return matrix
            return self.expected_chi().mean(axis = (1,2))
        
        else:
            # calcualte the over all mean
            return self.expected_chi().mean(axis = (0,1,2))
        
    def variances(self, each_sim = True):
        # calculate variance
        if each_sim == True:
            # calculate the variances of expected chi for each parameter set and t
            # return list
            return self.expected_chi().var(axis = (1,2))
        
        else:
            # calcualte the over all variances
            return self.expected_chi().var(axis = (0,1,2))

In [3]:
test = ExpectedChi()

In [6]:
test.signals()

array([[[[  3.17028131e+00,  -9.73672708e-01,  -8.18773232e-01,
           -1.82839328e+00,   1.76381444e+00],
         [ -4.07866062e+00,   3.42005451e+00,  -1.22411071e+00,
            7.77806100e-01,  -2.75572927e-01],
         [  2.33430015e+00,  -4.19314711e+00,  -9.72789984e-01,
           -1.08701623e+00,   1.72582175e+00],
         [ -1.63483187e+00,   8.39469007e-02,  -1.22335958e+00,
            4.81722350e-01,   1.48356746e+00]],

        [[ -1.87963541e+00,   2.28144427e+00,   1.83086911e+00,
            1.09126187e+00,   1.82950743e+00],
         [ -1.08052620e+00,  -4.11793357e-02,  -1.54761079e+00,
           -3.09890021e-01,   1.16941856e+00],
         [ -1.65707381e+00,  -1.11055040e+00,  -1.64875653e+00,
           -1.94162390e+00,  -1.61924134e+00],
         [  3.80021549e-01,  -1.66711299e+00,   8.37911883e-01,
            3.47944461e+00,   1.77865127e+00]],

        [[ -1.95473452e-01,  -1.48492098e+00,  -1.22460009e+00,
            3.29650166e+00,   2.54194232e-01

In [2]:
import numpy as np
from scipy.stats import norm

In [3]:
norm.pdf(0)

0.3989422804014327